In [4]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook
from bokeh.models import CategoricalColorMapper, ColumnDataSource, HoverTool
from bokeh.models.tiles import WMTSTileSource
from bokeh.plotting import figure, show
from bokeh.plotting.helpers import DEFAULT_PALETTE
import pyproj
import itertools
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

output_notebook()

Loading BokehJS ...

In [5]:
def normalize_street_address(address):
    address = address.str.upper()
    address = address.str.lstrip('0')
    address = address.str.replace(r'\s+', ' ')
    address = address.str.replace(r'[^A-Z0-9&/ ]', '')
    address = address.str.replace(r'\b(STREET|ST|ROAD|RD|DRIVE|DR|AVENUE|AVE?|F & O)\b', '')
    address = address.str.replace(r' +', ' ')
    address = address.str.replace(r' DET(ROIT)? ?(MI.*|\d+)', '')
    address = address.str.replace(r' (DET(ROIT)?|MI(CHIGAN)?)$', '')
    address = address.str.replace(r'[^A-Z0-9&/ ]', '')
    address = address.str.rstrip()
    address = address.str.replace(r' (AND|&&) | ?/ ?', ' & ')
    address = address.str.replace(r' (APT|APARTMENTS?|ROOM|FLAT|BLDG?|PLANT)( ?\w?\d+)?$', '')
    return address

def get_points():
    df_311 = pd.read_csv('data/detroit-311.csv')
    #df_311.head()
    points_311 = pd.DataFrame.from_dict({
        'Lat': df_311.lat,
        'Lon': df_311.lng,
        'StreetAddress': normalize_street_address(df_311.address.map(lambda x: x.upper().split(' DETROIT,')[0])),
        'Source': '311',
        'SourceId': df_311.ticket_id,
    })
    #points_311.head()

    df_violations = pd.read_csv('data/detroit-blight-violations.csv')
    #df_violations.head()
    address_lines = df_violations.ViolationAddress.map(lambda x: x.split('\n'))
    latlon = address_lines.map(lambda x: x[-1][1:-1].split(', '))
    points_violations = pd.DataFrame.from_dict({
        'Lat': latlon.map(lambda x: float(x[0])),
        'Lon': latlon.map(lambda x: float(x[1])),
        'StreetAddress': normalize_street_address(address_lines.map(lambda x: x[0])),
        'Source': 'violations',
        'SourceId': df_violations.TicketID,
    })
    points_violations.head()

    df_crime = pd.read_csv('data/detroit-crime.csv', dtype={'INCINO': 'str'})
    #df_crime.head()
    points_crime = pd.DataFrame.from_dict({
        'Lat': df_crime.LAT,
        'Lon': df_crime.LON,
        'StreetAddress': normalize_street_address(df_crime.ADDRESS),
        'Source': 'crime',
        'SourceId': df_crime.INCINO,
    })
    # XXX: Ignore incidents without coordinates.
    pred = points_crime.Lat.isnull() | points_crime.Lon.isnull()
    print('dropping', pred.sum(), 'incidents')
    points_crime = points_crime[~pred]
    #points_crime.head()
    print(points_crime.size, 'incidents left')

    df_demolition = pd.read_csv('data/detroit-demolition-permits.tsv', sep='\t')
    #df_demolition.head()
    # XXX: Ignore addresses without coordinates for now.
    pred = df_demolition.site_location.str.endswith('\n', na=True)
    print('dropping', pred.sum(), 'incidents')
    df_demolition = df_demolition[~pred]
    address_lines = df_demolition.site_location.map(lambda x: x.split('\n'))
    latlon = address_lines.map(lambda x: x[-1][1:-1].split(', '))
    points_demolition = pd.DataFrame.from_dict({
        'Lat': latlon.map(lambda x: float(x[0])),
        'Lon': latlon.map(lambda x: float(x[1])),
        'StreetAddress': normalize_street_address(df_demolition.SITE_ADDRESS),
        'Source': 'demolition',
        'SourceId': df_demolition.PERMIT_NO,
    })
    #points_demolition.head()
    print(points_demolition.size, 'incidents left')

    points = pd.concat([
        points_311,
        points_violations,
        points_crime,
        points_demolition,
    ])
    return points

In [6]:
points = get_points()

# XXX: Discard non-Detroit incidents.
pred = points.StreetAddress.str.contains(r'\bUSA\b')
print('dropping', pred.sum(), 'incidents')
points = points[~pred]
print(points.size, 'incidents left')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (6,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


dropping 59 incidents
599360 incidents left
dropping 817 incidents
31580 incidents left
dropping 50 incidents
2268110 incidents left


In [7]:
points.sample(n=20)

,Lat,Lon,Source,SourceId,StreetAddress
82722,42.413400,-83.142100,crime,1503280078.1,16500 STOEPEL
62470,42.333000,-83.031100,crime,1508080030.1,1400 FRANKLIN
155599,42.434912,-83.152970,violations,185336,3495 OUTER
203060,42.397837,-83.251401,violations,229699,11240 OUTER
91295,42.427922,-83.271830,violations,130049,23545 KRESS
55656,42.383040,-83.241974,violations,76515,12906 FIELDING
34442,42.330900,-83.131700,crime,1504110072.1,MICHIGAN & PARKINSON FRONT OF JORDANS FAMILY ...
301365,42.400387,-83.254142,violations,335151,21540 FENKELL
113316,42.420600,-83.142200,crime,1502200025.1,7000 SANTA CLARA
266060,42.420199,-83.052817,violations,300614,17191 FENELON


In [23]:
lonlat_proj = pyproj.Proj(init='epsg:4326')
xy_proj = pyproj.Proj(init='epsg:3857')

def latlon_to_xy(lat, lon):
    try:
        return pyproj.transform(lonlat_proj, xy_proj, lon, lat)
    except Exception:
        return (None, None)

def add_xy(df):
    df = df.copy()
    if df.empty:
        return df
    xy = df.apply(lambda row: latlon_to_xy(row.Lat, row.Lon), axis=1)
    df['X'] = xy.map(lambda row: row[0])
    df['Y'] = xy.map(lambda row: row[1])
    return df

def repeat_palette(palette, length):
    return list(itertools.islice(itertools.cycle(palette), length))

world_topo_map = WMTSTileSource(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
    attribution='Tiles &copy; <a href="https://services.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer">ArcGIS</a>'
)

In [9]:
lon = -83.2
lat = 42.4
radius = 1 / 512 # * 4
subset = points[
    (lon - radius < points.Lon) & (points.Lon < lon + radius) &
    (lat - radius < points.Lat) & (points.Lat < lat + radius)
]
print(len(subset), 'incidents')

510 incidents


In [10]:
def plot_points(points):
    p = figure()
    p.axis.visible = False

    p.add_tools(HoverTool(tooltips=[
        ('Address', '@StreetAddress'),
        ('Location', '(@Lat, @Lon)'),
        ('Incident', '@Source @SourceId'),
    ]))

    p.circle(
        source=ColumnDataSource(data=add_xy(points)), x='X', y='Y',
        size=9, fill_color='blue', fill_alpha=0.5, line_color=None,
    )

    p.add_tile(world_topo_map)

    show(p)
    
plot_points(subset)

In [11]:
def cluster(points):
    labels = DBSCAN(eps=0.00001, min_samples=1).fit_predict(points[['Lat', 'Lon']])
    labels = pd.Series(labels, index=points.index, name='Label')
    return labels

subset = subset.copy()
subset['Label'] = cluster(subset)
print(len(subset), 'incidents')
print(len(set(subset.Label)), 'buildings')

510 incidents
239 buildings


In [12]:
def plot_labeled(points):
    p = figure()
    p.axis.visible = False

    p.add_tools(HoverTool(tooltips=[
        ('Label', '@Label'),
        ('Address', '@StreetAddress'),
        ('Location', '(@Lat, @Lon)'),
        ('Incident', '@Source @SourceId'),
    ]))

    unique_labels = list(points.Label.unique())

    color_mapper = CategoricalColorMapper(
        factors=unique_labels,
        palette=repeat_palette(DEFAULT_PALETTE, len(unique_labels)),
    )

    p.circle(
        source=ColumnDataSource(data=add_xy(points)), x='X', y='Y',
        size=6,
        alpha=0.4,
        color=dict(field='Label', transform=color_mapper),
    )

    p.add_tile(world_topo_map)

    show(p)
    
plot_labeled(subset)

In [13]:
def build_classifier(points):
    #classifier = RadiusNeighborsClassifier(radius=0.00001, outlier_label=-1)
    classifier = KNeighborsClassifier(1)
    classifier.fit(points.as_matrix(columns=['Lat', 'Lon']), points.Label)
    return classifier

In [14]:
def plot_classifier(classifier, points):
    mesh_density = 300
    lat_min, lat_max = points.Lat.min(), points.Lat.max()
    lon_min, lon_max = points.Lon.min(), points.Lon.max()
    mesh_lats, mesh_lons = np.meshgrid(
        np.linspace(lat_min, lat_max, mesh_density),
        np.linspace(lon_min, lon_max, mesh_density),
    )
    mesh_labels = classifier.predict(np.c_[mesh_lats.ravel(), mesh_lons.ravel()])
    mesh_labels = mesh_labels.reshape(mesh_lats.shape)

    p = figure()
    p.axis.visible = False

    p.add_tools(HoverTool(tooltips=[
        ('Label', '@Label'),
        ('Address', '@StreetAddress'),
        ('Location', '(@Lat, @Lon)'),
        ('Incident', '@Source @SourceId'),
    ]))

    unique_labels = list(points.Label.unique())

    color_mapper = CategoricalColorMapper(
        factors=unique_labels,
        palette=repeat_palette(DEFAULT_PALETTE, len(unique_labels)),
    )

    points_xy = add_xy(points)
    x_min, x_max = points_xy.X.min(), points_xy.X.max()
    y_min, y_max = points_xy.Y.min(), points_xy.Y.max()
    p.image(
        image=[mesh_labels.T], x=x_min, y=y_min, dw=x_max - x_min, dh=y_max - y_min,
        alpha=0.1,
        color_mapper=color_mapper,
    )

    p.circle(
        source=ColumnDataSource(data=points_xy), x='X', y='Y',
        size=6,
        color=dict(field='Label', transform=color_mapper),
        line_color='black',
    )

    p.add_tile(world_topo_map)

    show(p)
    
subset_classifier = build_classifier(subset)
plot_classifier(subset_classifier, subset)

In [15]:
is_building = points.StreetAddress.str.contains(r'^\d+ ')
buildings = points[is_building]
print(len(buildings))
buildings.head()

420534


,Lat,Lon,Source,SourceId,StreetAddress
0,42.383998,-83.161039,311,1516722,1312013130 ILENE
1,42.440471,-83.080919,311,1525361,1485 E OUTER
2,42.445244,-82.962038,311,1525218,15460 EASTBURN
3,42.421043,-83.166194,311,1525214,17541 MENDOTA
5,42.399431,-83.158100,311,1525087,14902 KENTUCKY


In [17]:
buildings = buildings.copy()
buildings['Id'] = cluster(buildings)

In [25]:
buildings.sample(10)

,Lat,Lon,Source,SourceId,StreetAddress,Id
106030,42.438650,-83.004813,violations,128919,19536 HOOVER,63987
26827,42.389000,-83.273500,crime,1509010228.1,14200 RIVERVIEW,130838
7859,42.358840,-83.239563,violations,24145,8886 STOUT,21359
37119,42.331681,-83.047996,violations,58675,17200 MCNICHOLS,15462
203674,42.419213,-82.984761,violations,232872,12820 AUGUST,60472
65664,42.357800,-83.048400,crime,1511160070.1,1300 E CANFIELD,157792
37898,42.417216,-82.976830,violations,57084,13409 WILFRED,26707
182094,42.388980,-83.139533,violations,203836,13730 LIVERNOIS,37154
42891,42.434800,-82.994600,crime,1508270044.1,19100 WALTHAM,142165
156717,42.331681,-83.047996,violations,194341,2987 BEATRICE,15462


In [26]:
plot_labeled(buildings.sample(10000).rename(columns={'Id': 'Label'}))